In [2]:
from utils.sentiment_analysis import SentimentAnalysis
from utils.translator import Translator
from utils.scraper import call_url_get_bs4
from bs4 import BeautifulSoup
from requests.structures import CaseInsensitiveDict
import requests
import time
from datetime import datetime, timedelta

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\madsh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
cookies = {
    
}

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36", 
}

In [4]:
soup = call_url_get_bs4(f"https://api.borsen.dk/nyheder/side/1", cookies=cookies, headers=headers)



c:\Users\madsh\OneDrive\Code\Python\BiksStonks\data_collectors\python\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.borsen.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
for link in soup.find_all('li'):
    print(link.a.get('href'))

https://borsen.dk/nyheder/finans/bankinvest-piller-boreselskab-ud-af-baeredygtig-fond?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_1
https://borsen.dk/nyheder/finans/styrelse-opfordrer-banker-til-pristjek?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_2
https://borsen.dk/nyheder/udland/dansk-general-putin-sidste-losning-er-at-smadre-ukraine-fordi-hans-haer-med-amatorer-faar-taesk?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_3
https://borsen.dk/nyheder/markedsberetninger/aktier-i-europa-hm-og-s4-capital-traekker-ned-i-rodt-europaeisk-marked?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_4
https://borsen.dk/nyheder/pleasure/menneskeklog-film-om-sorg-savn-og-kaerlighed?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_5
https://borsen.dk/nyheder/baeredygtig/ny-chef-i-nemligcom-skal-saenke-indkobskurvens-fodaftryk?utm_source=seneste-nyt&utm_medium=row_1&utm_campaign=list_6
https://borsen.dk/nyheder/politik/forslag-til-dansk-nato-bidra

AttributeError: 'NoneType' object has no attribute 'get'